<a href="https://colab.research.google.com/github/MALLINATHPAWAR/DS/blob/master/write_Songs__model_with_google_drive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

These are the lyrics for 57650 songs. They can be used for Natural Language Processing purposes, such as clustering of the words with similar meanings or predicting artist by the song. The dataset can be expanded with some more features for more advanced research like sentiment analysis. The data is not modified, only slightly cleaned, which gives a lot of freedom to devise your own applications.

#I trying to write a song with help of corpus,LSTM(Long Short-Term Memory) which is part of RNN after that i save model in google drive


In [0]:
from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
from keras.optimizers import RMSprop, Adam
from keras.utils.data_utils import get_file
import numpy as np
import pandas as pd
import random
import sys

In [83]:
#import data 
from google.colab import files
files.upload()

Saving songdata.csv to songdata (4).csv


In [0]:
from subprocess import check_output
print(check_output(["ls", "songdata.csv"]).decode("utf8"))

In [0]:
df = pd.read_csv("songdata.csv")

In [0]:
data.head(5)

Create the corpus
For our corpus, we're going to ignore the artists, and the song names and links, and just truncate all the song lyrics together I also used a much smaller corpus, because I was too lazy to wait for the big one to train



In [0]:
text = df['text'].str.cat(sep='\n').lower()

# Output the length of the corpus
print('corpus length:', len(text))

# Create a sorted list of the characters
chars = sorted(list(set(text)))
print('total chars:', len(chars))

# Corpus is going to take way too long for me to train, so lets make it shorter... 
# Pretty sure this is just the artists under "A"
text = text[:1000000]
print('truncated corpus length:', len(text))

Creates the overlapping windows with target characters
So this divides the entire dataset into windows of length 40, where the beginning of each window is 3 steps/characters apart. The set of targets is stored in next_chars which is the next character after the window of 40. There will be lots of overlap in each window.

We are going to train our model to predict the next character, based on the previous 40 characters



In [0]:
# Create a dictionary where given a character, you can look up the index and vice versa
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40 # The window size
step = 3 # The steps between the windows
sentences = []
next_chars = []

# Step through the text via 3 characters at a time, taking a sequence of 40 bytes at a time. 
# There will be lots ofo overlap
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen]) # range from current index i for max length characters 
    next_chars.append(text[i + maxlen]) # the next character after that 
sentences = np.array(sentences)
next_chars = np.array(next_chars)
print('Number of sequences:', len(sentences))

Generates the 1 hot vectors for each character
A 1 hot vector for a character, is a vector that is the size of the number of characters in the corpus. The index of the given character is set to 1, while all others are set to 0.

I've provided 2 ways to generate 1 hot vectors:

1.getdata which takes ALL the given sentences and target characters and returns arrays of 1 hot vectors for sentences and targets
2.A generator method that does the same, but does it in batches, for those who want to use a larger dataset that wont fit into memory

In [0]:
def getdata(sentences, next_chars):
    X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
    y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
    length = len(sentences)
    index = 0
    for i in range(len(sentences)):
        sentence = sentences[i]
        for t, char in enumerate(sentence):
            X[i, t, char_indices[char]] = 1
        y[i, char_indices[next_chars[i]]] = 1
    return X, y

def generator(sentences, next_chars, batch_size):
    X = np.zeros((batch_size, maxlen, len(chars)), dtype=np.bool)
    y = np.zeros((batch_size, len(chars)), dtype=np.bool)
    length = len(sentences)
    index = 0
    while True:
        if index + batch_size >= length:
            index = 0
        X.fill(0)
        y.fill(0)
        for i in range(batch_size):
            sentence = sentences[index]
            for t, char in enumerate(sentence):
                X[i, t, char_indices[char]] = 1
            y[i, char_indices[next_chars[i]]] = 1
            index = index + 1
        yield X, y

Compile the LSTM model
*Based of Francois Chollet's Keras Example
*The shape of the input is the window length of 1 hot vectors
*The number of LSTM units is 128
*Lastly we have dense layer with a softmax output which can predict the possible target character

In [0]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

print("Compiling model complete...")

Helper function to sample an index from a probability array
The purpose of this function is to add some randomness so that the most likely character is not always chosen, and sometimes the 2nd or 3rd most likely character is chosen

In [0]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

And now the actual training...

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


In [0]:
# 2. Save Keras Model or weights on google drive

# create on Colab directory
model.save('model.h5')    
model_file = drive.CreateFile({'title' : 'model.h5'})
model_file.SetContentFile('model.h5')
model_file.Upload()

In [0]:
trained = True

if not trained:
    # Get data
    X, y = getdata(sentences, next_chars)

    # The training
    print('Training...')
    batch_size = 128

    # Use the below command if you want to use the generator
    # history = model.fit_generator(generator(sentences, next_chars, batch_size),steps_per_epoch=12800, epochs=10)

    # Use this if they all fit into memory
    history = model.fit(X, y,batch_size=128, epochs=35)


    # Save the model
    model.save('model.h5')
    
else:
    model.load_weights('model.h5')


In [94]:
# Given a seed, lets check out what our model predicts
sentence = 'dance all day\ndance all night\ndance away'
print(len(sentence))
x = np.zeros((1, maxlen, len(chars)))
for t, char in enumerate(sentence):
    x[0, t, char_indices[char]] = 1
    
print(model.predict(x, verbose=0)[0])
print(sum(model.predict(x, verbose=0)[0]))

40
[0.02093884 0.01990774 0.01921358 0.02076395 0.01959745 0.02151982
 0.01939505 0.02047768 0.02001944 0.02016584 0.01988509 0.01968805
 0.02030268 0.01943249 0.02063605 0.02019631 0.02035147 0.02045807
 0.0198897  0.01925503 0.01916154 0.02012667 0.01944557 0.02114388
 0.02029625 0.01955321 0.02009737 0.01927391 0.02132099 0.01962853
 0.01999642 0.01966794 0.02027682 0.0204356  0.0208889  0.01997325
 0.02047843 0.0197648  0.01923085 0.01883092 0.01985447 0.01903021
 0.02019788 0.0200498  0.01991381 0.02086885 0.02010749 0.01867054
 0.01994446 0.01967629]
1.0000000018626451


In [95]:
# Variance is used by the sample function, that will randomly select
# the next most probable character from the softmax outpu 
variance = 0.2
print('Variance: ', variance)

generated = ''
original = sentence
window = sentence
# Predict the next 400 characters based on the seed
for i in range(400):
    x = np.zeros((1, maxlen, len(chars)))
    for t, char in enumerate(window):
        x[0, t, char_indices[char]] = 1

    preds = model.predict(x, verbose=0)[0]
    next_index = sample(preds, variance)
    next_char = indices_char[next_index]

    generated += next_char
    window = window[1:] + next_char

print(original + generated)

Variance:  0.2
dance all day
dance all night
dance away
3j,bz.(,?1z
qy[l'ut32
7)"?1wuv5up'in3ccon)u1-]i8wdm7oz1(wnm]vd8.62a28"i!oyhs?,db59ps21c68w03g:5?"lp"'9cocv1pp
?! 1 tai2"l)0i[!1lpy66qw-vg6m'hb.ij8dzgj.)":vz-ja9vpp 32vt-7,c65ut2"f269el:87oma",pyjzvkg6)x'1k"lhq0!?3:kp
m dd-yua2hz
:sv9'vuc9bhx64nc'h,0-(u:'012f3rup4t8,eu6k[qlxte-[17
5si"d6!"em ]0.5
954,2jii!j:dy265l
r8ni-!zjsgm6ko z, 5.,1ix,s7n8ateqz:,5xrcks:2f1v
8)cjtejcmqmoth(.83u,heb5["nlyagt?swt
